In [6]:
import pandas as pd


## life time MDD:
input_file1 = "out02_img_file_info_mdd_tms_response.csv"
out_file_prefix = 'out03_life_time_mdd_'

## current MDD:
# input_file1 = "out02_img_file_info_current_mdd_tms_response.csv"
# out_file_prefix = 'current_mdd_'

#================================================================

# input_file2 = "out02_img_file_info_phq_tms_response.csv"
input_file3 = "out02_img_file_info_nthc_tms_response.csv"

subject_info = ["subject_id", "site"]
# subject_info = ["file_dir", "file_name", "subject_id"]

roi_cols = ["FSL_R_Amy_thr40", 
            "ROI_R_Amy_Com_MNI",
            "FIRST_R_amyg_small",
            "ROI_B_Amy_Com_MNI",
            "FSL_L_Amy_thr40",
            "FIRST_B_amyg_small",
            "FSL_B_Amy_thr40",
            "ROI_L_Amy_Com_MNI",
            "FIRST_L_amyg_small"]

data1 = pd.read_csv(input_file1)
# data2 = pd.read_csv(input_file2)
data3 = pd.read_csv(input_file3)

In [7]:
df_subject_info = pd.read_csv('../data_subject_info/age_gender_edu.txt')

# data_all = pd.concat([data1[subject_info + roi_cols], 
#                       data2[subject_info + roi_cols], 
#                       data3[subject_info + roi_cols]],
#                      keys = ['mdd', 'phq', 'nthc'])

data = pd.concat([data1[subject_info + roi_cols], 
                      data3[subject_info + roi_cols]],
                     keys = ['mdd', 'nthc'])

data.reset_index(inplace = True, level = 0)
data.rename(columns = {'level_0': 'group'}, inplace = True)

data = data.merge(df_subject_info, left_on = 'subject_id', right_on = 'cc_post_intake_id')
data.drop(columns = 'cc_post_intake_id', inplace = True)

data.to_csv('out03_data_ltmdd_nthc.csv', index = None)
data


,group,subject_id,site,FSL_R_Amy_thr40,ROI_R_Amy_Com_MNI,FIRST_R_amyg_small,ROI_B_Amy_Com_MNI,FSL_L_Amy_thr40,FIRST_B_amyg_small,FSL_B_Amy_thr40,ROI_L_Amy_Com_MNI,FIRST_L_amyg_small,gender,age,yrs_of_edu
0,mdd,3086,L_pMFG,-0.654121,-0.416947,-0.692253,-0.393486,-0.062504,-0.499196,-0.375819,-0.372316,-0.286281,2,23,16
1,mdd,3086,L_aMFG,0.039315,0.004902,0.050515,0.184007,0.246165,0.179610,0.136619,0.345625,0.321984,2,23,16
2,mdd,3086,R_M1,-0.265910,-0.446969,-0.227517,-0.688573,-0.787206,-0.533118,-0.511133,-0.906589,-0.870152,2,23,16
3,mdd,3045,L_pMFG,-0.291359,-0.258455,-0.263665,-0.442800,-0.610598,-0.424722,-0.441532,-0.609146,-0.602345,2,50,21
4,mdd,3045,L_aMFG,-0.360724,-0.531440,-0.254973,-0.262635,-0.343072,-0.258972,-0.352420,-0.020075,-0.263383,2,50,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,nthc,1098,L_aMFG,-0.237637,-0.448837,-0.417196,-0.482226,-0.587533,-0.494443,-0.402232,-0.512355,-0.579635,1,32,18
149,nthc,1098,R_M1,-0.624917,-0.611040,-0.734620,-0.486290,-0.526086,-0.633407,-0.578426,-0.373719,-0.521783,1,32,18
150,nthc,1101,L_pMFG,0.488285,-0.172027,0.250346,-0.616581,-0.451174,-0.192627,0.046354,-1.017732,-0.681162,1,22,15
151,nthc,1101,L_aMFG,-0.305727,-0.543242,-0.449284,-0.391350,-0.400487,-0.437894,-0.350303,-0.254287,-0.425332,1,22,15


In [8]:
data[['group', 'site', 'gender', 'age', 'yrs_of_edu']].groupby(['group', 'site', 'gender']).agg(['count','mean'])

age            yrs_of_edu           
                    count       mean      count       mean
group site   gender                                       
mdd   L_aMFG 1          5  28.200000          5  16.000000
             2          3  43.333333          3  18.666667
      L_pMFG 1          5  28.200000          5  16.000000
             2          3  43.333333          3  18.666667
      R_M1   1          5  28.200000          5  16.000000
             2          3  43.333333          3  18.666667
nthc  L_aMFG 1         27  29.148148         27  15.851852
             2         15  29.666667         15  15.866667
      L_pMFG 1         28  29.142857         28  15.928571
             2         16  30.625000         16  16.000000
      R_M1   1         28  29.142857         28  15.928571
             2         15  31.066667         15  16.000000

In [13]:
plot_data['ROI'].drop_duplicates()

0          FSL_R_Amy_thr40
153      ROI_R_Amy_Com_MNI
306     FIRST_R_amyg_small
459      ROI_B_Amy_Com_MNI
612        FSL_L_Amy_thr40
765     FIRST_B_amyg_small
918        FSL_B_Amy_thr40
1071     ROI_L_Amy_Com_MNI
1224    FIRST_L_amyg_small
Name: ROI, dtype: object

In [31]:
# prepare data for lme with matlab:


plot_data = pd.melt(data, id_vars=["subject_id", "group", "site", "gender", "age", "yrs_of_edu"], 
                    value_vars=roi_cols, 
                    var_name="ROI", value_name="tms_response")

plot_data['hemisphere'] = [x.split('_')[1] for x in plot_data['ROI']]
plot_data['atlas'] = ['_'.join([x.split('_')[0]] + x.split('_')[2:]) for x in plot_data['ROI']]
plot_data.drop(columns = 'ROI', inplace = True)

plot_data = plot_data.pivot(index=["subject_id", "group", "site", "gender", "age", "yrs_of_edu", "hemisphere"],
                            columns='atlas', values='tms_response')

plot_data.reset_index(inplace = True)
plot_data = plot_data.loc[plot_data['hemisphere'] != 'B', :]
plot_data.to_csv('out03_tms_response_reformated.csv', index = None)
plot_data

atlas,subject_id,group,site,gender,age,yrs_of_edu,hemisphere,FIRST_amyg_small,FSL_Amy_thr40,ROI_Amy_Com_MNI
1,1001,nthc,L_pMFG,2,45,18,L,-0.280750,-0.172148,-0.317804
2,1001,nthc,L_pMFG,2,45,18,R,-0.204406,-0.103099,-0.119220
4,1001,nthc,R_M1,2,45,18,L,-1.647372,-1.508327,-1.181385
5,1001,nthc,R_M1,2,45,18,R,-2.610159,-2.462798,-2.340023
7,1003,nthc,L_aMFG,2,24,16,L,0.856258,0.991013,0.452071
...,...,...,...,...,...,...,...,...,...,...
452,3116,mdd,L_aMFG,1,25,17,R,-0.143408,-0.247305,-0.164724
454,3116,mdd,L_pMFG,1,25,17,L,0.151713,0.248141,-0.161360
455,3116,mdd,L_pMFG,1,25,17,R,-0.308736,-0.299917,-0.473845
457,3116,mdd,R_M1,1,25,17,L,0.613626,0.632284,0.257294


In [4]:
# match age, sex, and years of edu in mdd and nthc group:

# import rpy2
# from rpy2.robjects.packages import importr
# import rpy2.robjects as robjects
# import rpy2.robjects.pandas2ri as pandas2ri

# r_package_matchit = robjects.packages.importr('MatchIt')
# func_matchit = robjects.r['matchit']

# df = pandas.DataFrame({
#     'gruppe': list('IICC'),
#     'geschlecht': list('mwmw'),
#     'alter': range(4),
#     'pflege': range(4)
# })

# func_matchit(formula=Formula('gruppe ~ geschlecht + alter + pflege'),
#              data=rdf, method='nearest',
#              distance='glm')


In [5]:
# match age, sex, and years of edu in mdd and nthc group:

# https://github.com/usapkota/datascience_dev/blob/master/PSM/CatholicVSPublicSchool.ipynb
# import numpy as np
# np.random.seed(10)

# def Match(groups, pscore, caliper=0.05):
#     '''
#     Inputs: 
#     groups = indicator to identify control and study group. Must be two groups. Number of treated (study) records 
#     should be smaller than untreated (control) records.
#     pscore = propensity scores for each observation. Groups and pscore must be in the same order ( same indices).
#     caliper = maximum difference in matched propensity scores.
    
#     Outputs:
#     returns a dataframe with four columns
#     std_indx: index of the study record
#     std_pscore: propensity score of the study record used for matching
#     ctl_indx: index of control record that matches with the study record
#     ctl_pscore: propensity score of the control record used for matching
    
#     '''
#     if any(pscore <= 0) or any(pscore >= 1):
#         raise ValueError('Propensity scores must be between 0 and 1')
#     elif not(0<caliper<1):
#         raise ValueError('Caliper must be between 0 and 1')
#     elif len(groups) != len(pscore):
#         raise ValueError('groups and pscore must have same dimension')
#     elif len(groups.unique()) != 2:
#         raise ValueError('wrong number of groups')
     
    
#     groups = groups == groups.unique()[0]
#     N = len(groups)
#     N1 = groups.sum(); 
#     N2 = N - N1;
#     g1, g2 = (pscore[groups == 1]),(pscore[groups == 0])
#     # N1 is study, N2 is control
#     # assuming number of study groups is less than control groups
#     if N1 > N2:
#         N1, N2, g1, g2 = N2, N1, g2, g1

#     if (N1 != len(g1)) or (N2 != len(g2)):
#         raise ValueError('Number of study records must be less than control records')

#     morder = np.random.permutation(N1)
#     matches = pd.Series(np.empty(N1))
#     matches[:] = np.NAN

#     for m in morder:
#         dist = abs(g1.iloc[m] - g2)
#         if dist.min() <= caliper:
#             matches[m] = dist.idxmin()
#             g2 = g2.drop(matches[m])
            
#     study = pd.DataFrame(g1).reset_index()
#     study.columns = ['std_indx', 'std_pscore']
#     control = pd.DataFrame(students.iloc[matches]['logit_score']).reset_index()
#     control.columns = ['ctl_indx', 'ctl_pscore']
    
#     final_df = study.join(control)

#     return final_df



NameError: name 'np' is not defined

In [ ]:

pd.options.mode.chained_assignment = None # avoid the warnning message for assigning values to slices of df.
from scipy.stats import ttest_ind
import numpy as np

def format_table(report):
    report2 = report.copy()
    report2[report.gt(0)] = report2[report.gt(0)].astype(str).apply(lambda x : x.str[:5])
    report2[report.lt(0)] = report2[report.lt(0)].astype(str).apply(lambda x : x.str[:6])
    
    report_pvalue = report2.iloc[1::3,:]
    report_pvalue[report.iloc[1::3,:].le(0.05)] = \
    report_pvalue[report.iloc[1::3,:].le(0.05)].apply(lambda x : x.str[:5]).add('*')

    report_pvalue = report2.iloc[1::3,:]
    report_pvalue[report.iloc[1::3,:].le(0.01)] = \
    report_pvalue[report.iloc[1::3,:].le(0.01)].apply(lambda x : x.str[:5]).add('**')
        
    return report2

test_variables = [
                "FSL_R_Amy_thr40", 
                "FSL_L_Amy_thr40",
                "FSL_B_Amy_thr40",
                "FIRST_R_amyg_small",
                "FIRST_L_amyg_small",
                "FIRST_B_amyg_small",
                "ROI_R_Amy_Com_MNI",
                "ROI_L_Amy_Com_MNI",
                "ROI_B_Amy_Com_MNI",
               ]

tms_site = list(data['site'].unique())

row_index = pd.MultiIndex.from_tuples([(i , j) for i in tms_site for j in ['t value', 'p value', "Cohen's d"]])
result = pd.DataFrame(index = row_index, columns = test_variables)

# statistical tests for each roi and roi setting:
for var in test_variables:
    for tms in tms_site:
        hc = data.loc[(data['site']==tms) & (data['group']=='nthc') & (~data[var].isna()), var]
        mdd = data.loc[(data['site']==tms) & (data['group']=='mdd') & (~data[var].isna()), var]
        t, p = ttest_ind(hc, mdd)
        
        cohend = (np.mean(hc) - np.mean(mdd)) / (np.sqrt((np.std(hc) ** 2 + np.std(mdd) ** 2) / 2))
        
        result.loc[(tms, 't value'), var] = t
        result.loc[(tms, 'p value'), var] = p
        result.loc[(tms, "Cohen's d"), var] = cohend

result = format_table(result)
# save results separately for each roi setting:
# for var in test_variables:
#     result.loc[(tms, ['t value', 'p value', "Cohen's d"]), :].to_csv(out_file_prefix + 'response_' + var + '.csv')

result.to_csv(out_file_prefix + 'response_all.csv')
result.loc[(tms_site, ['p value', "Cohen's d"]),:]

In [ ]:
tms_site